In [5]:
import gspread
from gspread_dataframe import get_as_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe
import pyodbc
import pandas as pd
import numpy as np

# Setup
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("python-459011-73acbdff4227.json", scope)
client = gspread.authorize(creds)


# Open Sheet
sheet = client.open_by_key("1lcZG8UWOMmbAgC_x0RtMSed_PM1Z2h_HpC-5GUSotVc")
Data_Sheet = sheet.worksheet("ExportUserFeedback")


# SQL Server connection details
server = '157.119.230.120,4071'     # e.g., 'localhost'
database = 'PowerBI_LISV2'       # e.g., 'PowerBI_LISV2'
username = 'sa'
password = 'Y0m@Sql.!.123'

# Connection string
conn_str = f"""
    DRIVER={{ODBC Driver 17 for SQL Server}};
    SERVER={server};
    DATABASE={database};
    UID={username};
    PWD={password};
    TrustServerCertificate=yes;
"""

# Connect and fetch into DataFrame
try:
    conn = pyodbc.connect(conn_str)
    print("✅ Connected to SQL Server")

    # Define your query
    query = '''
               SELECT distinct a.[VisitDate&Time]
                             ,a.[Month]
                             ,a.[Year]
                             ,a.[Region]
                             ,a.[State]
                             ,a.[City]
                             ,a.[ISPCode]
                             ,a.[UserName]
                             ,a.[FullName]
                             ,a.[Designation]
                             ,a.[ClientStoreCode]
                             ,a.[LISStoreCode]
                             ,a.[StoreName]
                             ,a.[SectionCode]
                             ,a.[SectionName]
                             ,a.[QRCode]
                             ,a.[QRName]
                             ,a.[FormId]
                             ,a.[Question]
                             ,a.[Response]
							 ,b.[TimeDuration]
   
             FROM [PowerBI_LISV2].[BeamSuntoryManagement].[UserResponseExport] a
			 LEFT JOIN [PowerBI_LISV2].[BeamSuntoryManagement].[ISPStoreAttendance_NormalExport] b
			 ON left(a.[VisitDate&Time],10)=left(b.[VisitDate],10)
			 AND  a.[UserName]=b.[UserName]
			 AND a.[LISStoreCode]=b.[LISStoreCode]
             WHERE a.MONTH = 10
             
            
    '''

    # Fetch results into a DataFrame
    Data = pd.read_sql(query, conn)

    # Close the connection
    conn.close()



except Exception as e:
    print(f"❌ Error: {e}")



    
Data_Sheet.batch_clear(["A:U"])
set_with_dataframe(Data_Sheet, Data)


✅ Connected to SQL Server


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_17860\2350070601.py:76: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Data = pd.read_sql(query, conn)
